In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV, cross_validate
from time import time
import matplotlib.pyplot as plt
from Loader import train_data, test_data
from matplotlib import colors
from sklearn.metrics import accuracy_score

In [3]:
df_train = train_data()
print(df_train.shape)
df_train.head(3)

(100000, 3073)


,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,label
0,114,112,113,119,117,118,117,117,115,111,...,100,91,86,99,91,88,98,90,87,Fake
1,206,204,205,215,213,214,224,222,223,228,...,126,125,123,124,123,121,123,122,120,Fake
2,153,185,208,130,162,185,153,185,208,162,...,167,178,172,166,177,171,165,176,170,Fake


In [4]:
df_sample = df_train.sample(frac=0.1, random_state=0)
print(df_sample.shape)
X = df_sample.drop('label', axis=1)
X = (X-X.mean())/X.std()
y = df_sample.label

(10000, 3073)


In [6]:
df_test = test_data()
print(df_test.shape)
df_test.head(3)

(20000, 3073)


,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,label
0,11,45,54,14,48,57,20,52,63,26,...,59,71,71,60,72,72,46,58,58,Fake
1,86,84,85,60,58,59,56,54,55,87,...,61,59,60,57,55,56,59,57,58,Fake
2,96,80,90,97,81,92,102,86,97,110,...,75,65,66,74,64,62,70,61,56,Fake


In [7]:
X_test = df_test.drop('label',axis=1)
X_test=(X_test - X_test.mean())/X_test.std()
y_true = df_test.label

In [8]:
lgr = LogisticRegression(n_jobs=-1)
lgr.fit(X,y)

LogisticRegression(n_jobs=-1)

In [10]:
cv_results = cross_validate(lgr, X, y, return_train_score=True, cv=5)
print('Validation Accuracy:', cv_results['test_score'].mean())
print('Train Accuracy:', cv_results['train_score'].mean())

Validation Accuracy: 0.6312
Train Accuracy: 0.7390000000000001


In [ ]:
y_pred = lgr.predict(X_test)
print('Test Accuracy:', accuracy_score(y_true, y_pred))

In [26]:
# reg_stren = [0.0005, 0.001, 0.005]
# reg_stren = np.linspace(0.0001, 0.001, 5)
reg_stren = np.linspace(0.0001, 0.001, 10)
print(reg_stren)
pen = ['l2']
# solv = ['saga']
# iter = [10000]
# grid = {'C':reg_stren, 'penalty':pen, 'solver':solv, 'max_iter':iter}
grid = {'C':reg_stren, 'penalty':pen}

[0.0001 0.0002 0.0003 0.0004 0.0005 0.0006 0.0007 0.0008 0.0009 0.001 ]


In [27]:
time_start = time()

lgr = LogisticRegression(n_jobs = -1)
lgrCV = GridSearchCV(lgr, param_grid=grid, return_train_score=True, n_jobs=-1)
lgrCV.fit(X,y)

time_stop = time()
print('Comp Time:', (time_stop-time_start)/60, 'min')

/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
IOStream.flush timed out


Comp Time: 23.24534053405126 min


In [28]:
print('Hyper Parameters:', lgrCV.best_params_)

Hyper Parameters: {'C': 0.00039999999999999996, 'penalty': 'l2'}
R2 0.6635


In [58]:
# lgr = LogisticRegression(C=lgrCV.best_params_['C'],
#                          penalty=lgrCV.best_params_['penalty'],
#                          solver=lgrCV.best_params_['solver'],
#                          max_iter=lgrCV.best_params_['max_iter'])
lgr = LogisticRegression(C=lgrCV.best_params_['C'],
                         penalty=lgrCV.best_params_['penalty'])
lgr.fit(X, y)

LogisticRegression(C=0.00039999999999999996)

In [59]:
ix = lgrCV.cv_results_['mean_test_score'].argmax()

In [60]:
accuracy_train = lgrCV.cv_results_['mean_train_score'][ix].round(4)
accuracy_valid = lgrCV.cv_results_['mean_test_score'].max().round(4)

In [61]:
print('Training Accuracy:', accuracy_train)
print()
print('Validation Accuracy:', accuracy_valid)

Training Accuracy: 0.6946

Validation Accuracy: 0.6635


In [62]:
y_pred = lgr.predict(X_test)
print('Test Score:', accuracy_score(y_true, y_pred))

Test Score: 0.66575
